In [57]:
import pandas as pd
import numpy as np
import spacy

In [58]:
df = pd.read_csv("data/cleaned_data.csv")
df.head()

C:\Users\shrey\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,original text,cleaned text,joke
0,0,I am soooo glad I'm not circumcised! My corona...,soooo glad circumcised corona covered foreskin...,1.0
1,1,Did you know Google now has a platform for rec...,know google platform recording bowel movements...,1.0
2,2,What is the difference between my wife and my ...,difference wife vacuum vacuum nt snore sex amp...,1.0
3,3,What did the French man say to the attractive ...,french man say attractive teletubby oo lala,1.0
4,4,"Yo Mama Yo momma's so fat, that when she went ...",yo mama yo momma fat went zoo hippos got jealous,1.0


In [59]:
df = df.drop(columns=df.columns.values[0:2])

In [60]:
df.head()


,cleaned text,joke
0,soooo glad circumcised corona covered foreskin...,1.0
1,know google platform recording bowel movements...,1.0
2,difference wife vacuum vacuum nt snore sex amp...,1.0
3,french man say attractive teletubby oo lala,1.0
4,yo mama yo momma fat went zoo hippos got jealous,1.0


In [61]:
df.isnull().sum()


cleaned text    4546
joke              48
dtype: int64

In [62]:
df = df.dropna()
df.isnull().sum()

cleaned text    0
joke            0
dtype: int64

In [63]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [64]:
# Removes non-alphabetic characters
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['cleaned text'])

In [55]:
# Taking advantage of spaCy .pipe() attribute to speed-up the cleaning process
import re
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]
print(txt.type)

In [65]:
df_clean = pd.DataFrame({'clean': txt})
df_clean.head()
# df_clean = df_clean.dropna().drop_duplicates()
# df_clean.shape

,clean
